In [1]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

library("car")
require("lattice")
require("ez")

library("cowplot")
library("ggbeeswarm")

library("nortest")

library("effectsize")

theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))


options(width=240)
library(tidyverse)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'KR', 'S', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- NEWS: library('emmeans') now needs to be called explicitly!
- Get and set global pac

In [2]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")


Attaching package: ‘IRdisplay’


The following object is masked from ‘package:xtable’:

    display


Loading required package: papaja

Loading required package: tinytex



In [3]:
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}

In [4]:
AOV4 <- function(DATA,INFO){

    INFO <- paste("`",INFO,"`",sep="")
    DATA_SUMMARY <- DATA %>%
        group_by(cond0) %>%
        summarise(
            N = n(),
            valX_M = mean(valX),
            valX_SD = sd(valX),
            valX_SE = valX_SD / sqrt(N),
            .groups = "drop"
        )    
    DATA_aov <- aov_4(
        formula  <- FORMULA,
        data     <- DATA,
    )    
    DATA_aov_APA <- apa_print(DATA_aov)
    
    DATA_aov_EMM <- emmeans(DATA_aov, specs<-c("cond0","chan0"))
    # DATA_aov_EMM <- emmeans(DATA_aov, specs<-pairwise~cond0|chan0)
    # DATA_aov_EMM <- emmeans(DATA_aov, specs<-pairwise~cond0:chan0)
    DATA_aov_EMM_APA <- apa_print(DATA_aov_EMM)
    
    # DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM)
    # DATA_aov_EMM_PAIRS <- pairs(DATA_aov_EMM )

    # cont <- "trt.vs.ctrl1"
    # cont <- "trt.vs.ctrl"
    # cont <- "consec"
    # cont <- "poly"
    # cont <- "trt.vs.ctrlk"
    cont <- "pairwise"
    # DATA_aov_EMM_PAIRS <- contrast(DATA_aov_EMM, cont, by = c("chan0"))
    DATA_aov_EMM_PAIRS <- contrast(DATA_aov_EMM, cont, by = c("chan0"))
    
    
    DATA_aov_EMM_PAIRS_APA <- apa_print(DATA_aov_EMM_PAIRS)    

    return(list("info"      = INFO,
                "summary"   = DATA_SUMMARY,
                "aov"       = DATA_aov,
                "aov_APA"   = DATA_aov_APA,
                "emm"       = DATA_aov_EMM,
                "emm_APA"   = DATA_aov_EMM_APA,
                "pairs"     = DATA_aov_EMM_PAIRS,
                "pairs_APA" = DATA_aov_EMM_PAIRS_APA
               )
          )
}

# Peaks 0

In [5]:
ifName="./data/JOB_004/exports/df3_peaks3_crude.csv"


df0 <- read.csv(file = ifName)
df0 <- subset(df0, CHAN_BUND=="" | is.na(CHAN_BUND)  )
df0 <- subset(df0, quest0=="word_set" )

In [6]:
str(df0)


'data.frame':	15120 obs. of  15 variables:
 $ evoked0  : Factor w/ 1 level "evoked3": 1 1 1 1 1 1 1 1 1 1 ...
 $ quest0   : Factor w/ 2 levels "word_len","word_set": 2 2 2 2 2 2 2 2 2 2 ...
 $ cond0    : Factor w/ 8 levels "7","8","9","all_in",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ chan0    : Factor w/ 42 levels "AF3","AF4","C1",..: 26 26 26 26 26 26 26 26 26 26 ...
 $ tmin0    : num  0.1 0.1 0.19 0.19 0.35 0.35 0.41 0.41 0.6 0.6 ...
 $ tmax0    : num  0.19 0.19 0.29 0.29 0.45 0.45 0.58 0.58 0.895 0.895 ...
 $ mode0    : Factor w/ 2 levels "neg","pos": 2 1 2 1 2 1 2 1 2 1 ...
 $ chanX    : Factor w/ 42 levels "AF3","AF4","C1",..: 26 26 26 26 26 26 26 26 26 26 ...
 $ latX     : num  0.105 0.14 0.205 0.255 0.45 0.35 0.475 0.425 0.66 0.7 ...
 $ valX     : num  0.144 -2.242 0.216 -0.828 -0.886 ...
 $ SUB      : Factor w/ 42 levels "01mkxf","02ppxm",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ SES      : Factor w/ 1 level "eeg001": 1 1 1 1 1 1 1 1 1 1 ...
 $ TASK     : Factor w/ 1 level "lexdec": 1 1 1 1 1 1 1 1

In [7]:
df1 <- data.frame(df0)

In [8]:
df1 <- subset(df1, cond0 %in% c("noun_F","noun_M","verb_X") )
# df1 <- subset(df1, cond0 %in% c("noun_X","verb_X") )

In [9]:
'%!in%' <- function(x,y)!('%in%'(x,y))

# df1 <- subset(df1, SUB %!in% c("01mkxf","02ppxm","09msxf","15agxf","17makf","24jkxf","26mwxf","31ltxf","33kkxm","37lpxm") )

In [10]:
df1 <- droplevels(df1)
# df1 <- subset(df1, select = -c(evoked0,SES,TASK,RUN,chanX,quest0,CHAN_BUND) )
df1 <- subset(df1, select = c(cond0,chan0,tmin0,tmax0,mode0,valX,SUB) )


In [11]:
str(df1)

'data.frame':	9072 obs. of  7 variables:
 $ cond0: Factor w/ 3 levels "noun_F","noun_M",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ chan0: Factor w/ 6 levels "LC","LF","LP",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ tmin0: num  0.1 0.1 0.19 0.19 0.35 0.35 0.41 0.41 0.6 0.6 ...
 $ tmax0: num  0.19 0.19 0.29 0.29 0.45 0.45 0.58 0.58 0.895 0.895 ...
 $ mode0: Factor w/ 2 levels "neg","pos": 2 1 2 1 2 1 2 1 2 1 ...
 $ valX : num  0.0756 -2.2746 -0.0233 -1.0445 -1.3524 ...
 $ SUB  : Factor w/ 42 levels "01mkxf","02ppxm",..: 1 1 1 1 1 1 1 1 1 1 ...


In [12]:
as.matrix(summary(df1["cond0"]))

    cond0     
 noun_F:3024  
 noun_M:3024  
 verb_X:3024  

In [13]:
c(unique(df1["chan0"]))

$chan0
[1] LF RF LC RC LP RP
Levels: LC LF LP RC RF RP

In [14]:
c(unique(df1["tmin0"]))

$tmin0
[1] 0.10 0.19 0.35 0.41 0.60 0.11

https://www.rdocumentation.org/packages/afex/versions/0.27-2/topics/aov_car

In [15]:
# FORMULA  <- valX~(cond0|SUB)
# TEST     <- adjusted("free")

FORMULA  <- valX~(cond0*chan0|SUB)
TEST     <- adjusted("free")

# Produce all possible condition combinations

In [16]:
CONDS <- c()
mode0 <- as.matrix(unique(df1["mode0"]))
tmin0 <- as.matrix(unique(df1["tmin0"]))
for(tmin1 in tmin0 ){
    for(mode1 in mode0 ){
        CONDS <- c(CONDS,str_c("tmin0==",tmin1," & mode0==\"",mode1,"\""))
    }
}

In [17]:
length(CONDS)

[1] 12

## Example condition defining string

In [18]:
IDX=0
CONDS[1]

[1] "tmin0==0.1 & mode0==\"pos\""

In [19]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); 
data1 <- data1[complete.cases(data1),]; 
done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); 
display_markdown(done1$info); 
display_markdown("### Table"); 
done1$aov_APA$table; 
display_markdown("### Result"); 
display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); 
display_markdown("### Summary (all done1, incl. NANs)"); 
done1$summary; 
display_markdown("### Expected marginal means"); 
done1$emm_APA$table; 
display_markdown("### Paired comparisons for expected marginal means"); 
done1$pairs_APA$table

# Condition 1 of 12

`tmin0==0.1 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.42,1.78,73.06,0.32,.102,.001
2,Chan0,68.11,2.00,81.84,18.67,< .001,.556
3,Cond0 $\times$ Chan0,1.13,5.33,218.68,0.49,.343,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,2.167381,2.451860,0.1544526
noun_M,252,2.066938,2.468749,0.1555166
verb_X,252,2.090008,2.471338,0.1556797


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,1.13,"$[0.62$, $1.63]$",4.41,< .001
noun_M_LC,,noun_M,0.95,"$[0.45$, $1.45]$",3.71,< .001
verb_X_LC,,verb_X,0.98,"$[0.47$, $1.48]$",3.82,< .001
noun_F_LF,LF,noun_F,1.08,"$[0.58$, $1.58]$",4.22,< .001
noun_M_LF,,noun_M,0.78,"$[0.28$, $1.29]$",3.06,.002
verb_X_LF,,verb_X,0.84,"$[0.33$, $1.34]$",3.27,.001
noun_F_LP,LP,noun_F,4.06,"$[3.55$, $4.56]$",15.86,< .001
noun_M_LP,,noun_M,4.20,"$[3.69$, $4.70]$",16.40,< .001
verb_X_LP,,verb_X,4.02,"$[3.51$, $4.52]$",15.70,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.18,"$[-0.09$, $0.44]$",1.59,.250
LC_noun_F_verb_X,,noun_F - verb_X,0.15,"$[-0.11$, $0.41]$",1.35,.370
LC_noun_M_verb_X,,noun_M - verb_X,-0.03,"$[-0.29$, $0.24]$",-0.24,.967
LF_noun_F_noun_M,LF,noun_F - noun_M,0.30,"$[0.03$, $0.56]$",2.65,.022
LF_noun_F_verb_X,,noun_F - verb_X,0.24,"$[-0.02$, $0.51]$",2.16,.079
LF_noun_M_verb_X,,noun_M - verb_X,-0.05,"$[-0.32$, $0.21]$",-0.49,.877
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.14,"$[-0.40$, $0.12]$",-1.25,.422
LP_noun_F_verb_X,,noun_F - verb_X,0.04,"$[-0.22$, $0.30]$",0.36,.931
LP_noun_M_verb_X,,noun_M - verb_X,0.18,"$[-0.08$, $0.44]$",1.61,.241


In [20]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 2 of 12

`tmin0==0.1 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.70,1.86,76.41,0.33,.191,.001
2,Chan0,20.45,2.08,85.11,16.93,< .001,.274
3,Cond0 $\times$ Chan0,1.80,4.88,200.20,0.48,.116,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-1.735705,1.848611,0.1164516
noun_M,252,-1.682084,1.880921,0.1184869
verb_X,252,-1.645735,1.869333,0.1177569


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,-1.03,"$[-1.52$, $-0.54]$",-4.16,< .001
noun_M_LC,,noun_M,-1.19,"$[-1.68$, $-0.70]$",-4.80,< .001
verb_X_LC,,verb_X,-0.99,"$[-1.47$, $-0.50]$",-3.98,< .001
noun_F_LF,LF,noun_F,-2.78,"$[-3.26$, $-2.29]$",-11.21,< .001
noun_M_LF,,noun_M,-2.75,"$[-3.24$, $-2.26]$",-11.11,< .001
verb_X_LF,,verb_X,-2.71,"$[-3.20$, $-2.22]$",-10.94,< .001
noun_F_LP,LP,noun_F,-2.06,"$[-2.55$, $-1.58]$",-8.33,< .001
noun_M_LP,,noun_M,-2.00,"$[-2.49$, $-1.51]$",-8.08,< .001
verb_X_LP,,verb_X,-2.05,"$[-2.53$, $-1.56]$",-8.27,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.16,"$[-0.10$, $0.41]$",1.46,.312
LC_noun_F_verb_X,,noun_F - verb_X,-0.04,"$[-0.30$, $0.21]$",-0.40,.917
LC_noun_M_verb_X,,noun_M - verb_X,-0.20,"$[-0.46$, $0.05]$",-1.86,.153
LF_noun_F_noun_M,LF,noun_F - noun_M,-0.02,"$[-0.28$, $0.23]$",-0.23,.972
LF_noun_F_verb_X,,noun_F - verb_X,-0.07,"$[-0.32$, $0.19]$",-0.62,.812
LF_noun_M_verb_X,,noun_M - verb_X,-0.04,"$[-0.30$, $0.21]$",-0.39,.920
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.06,"$[-0.32$, $0.19]$",-0.58,.832
LP_noun_F_verb_X,,noun_F - verb_X,-0.02,"$[-0.27$, $0.24]$",-0.15,.988
LP_noun_M_verb_X,,noun_M - verb_X,0.05,"$[-0.21$, $0.30]$",0.43,.902


In [21]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 3 of 12

`tmin0==0.19 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.95,1.99,81.78,0.30,.149,.000
2,Chan0,13.67,1.99,81.76,23.83,< .001,.205
3,Cond0 $\times$ Chan0,3.73,6.07,248.75,0.42,.001,.004


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,1.964146,2.010653,0.1266593
noun_M,252,1.963329,2.122907,0.1337306
verb_X,252,1.880297,2.022787,0.1274236


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,1.58,"$[1.02$, $2.14]$",5.53,< .001
noun_M_LC,,noun_M,1.61,"$[1.05$, $2.17]$",5.64,< .001
verb_X_LC,,verb_X,1.56,"$[1.00$, $2.13]$",5.49,< .001
noun_F_LF,LF,noun_F,2.13,"$[1.57$, $2.69]$",7.47,< .001
noun_M_LF,,noun_M,1.95,"$[1.39$, $2.51]$",6.86,< .001
verb_X_LF,,verb_X,1.91,"$[1.35$, $2.47]$",6.70,< .001
noun_F_LP,LP,noun_F,1.95,"$[1.39$, $2.51]$",6.86,< .001
noun_M_LP,,noun_M,2.36,"$[1.80$, $2.92]$",8.29,< .001
verb_X_LP,,verb_X,1.89,"$[1.33$, $2.45]$",6.63,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.03,"$[-0.29$, $0.23]$",-0.28,.957
LC_noun_F_verb_X,,noun_F - verb_X,0.01,"$[-0.25$, $0.27]$",0.10,.995
LC_noun_M_verb_X,,noun_M - verb_X,0.04,"$[-0.22$, $0.31]$",0.38,.923
LF_noun_F_noun_M,LF,noun_F - noun_M,0.17,"$[-0.09$, $0.43]$",1.54,.274
LF_noun_F_verb_X,,noun_F - verb_X,0.22,"$[-0.05$, $0.48]$",1.94,.129
LF_noun_M_verb_X,,noun_M - verb_X,0.04,"$[-0.22$, $0.31]$",0.40,.915
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.41,"$[-0.67$, $-0.14]$",-3.64,.001
LP_noun_F_verb_X,,noun_F - verb_X,0.07,"$[-0.20$, $0.33]$",0.60,.818
LP_noun_M_verb_X,,noun_M - verb_X,0.47,"$[0.21$, $0.74]$",4.24,< .001


In [22]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

Warning message in summary.Anova.mlm(x$Anova):
“HF eps > 1 treated as 1”


# Condition 4 of 12

`tmin0==0.19 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,2.60,1.99,81.43,0.36,.081,.001
2,Chan0,11.47,2.00,81.90,22.79,< .001,.176
3,Cond0 $\times$ Chan0,3.05,6.18,253.30,0.45,.006,.003


...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-1.401727,1.977839,0.1245922
noun_M,252,-1.424256,2.024073,0.1275046
verb_X,252,-1.309952,1.960087,0.1234739


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,-0.51,"$[-1.06$, $0.04]$",-1.81,.071
noun_M_LC,,noun_M,-0.66,"$[-1.21$, $-0.11]$",-2.35,.020
verb_X_LC,,verb_X,-0.44,"$[-0.99$, $0.12]$",-1.55,.122
noun_F_LF,LF,noun_F,-1.78,"$[-2.33$, $-1.22]$",-6.32,< .001
noun_M_LF,,noun_M,-1.84,"$[-2.40$, $-1.29]$",-6.57,< .001
verb_X_LF,,verb_X,-1.79,"$[-2.35$, $-1.24]$",-6.39,< .001
noun_F_LP,LP,noun_F,-2.23,"$[-2.79$, $-1.68]$",-7.96,< .001
noun_M_LP,,noun_M,-1.88,"$[-2.44$, $-1.33]$",-6.71,< .001
verb_X_LP,,verb_X,-2.10,"$[-2.66$, $-1.55]$",-7.50,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.15,"$[-0.13$, $0.43]$",1.28,.405
LC_noun_F_verb_X,,noun_F - verb_X,-0.07,"$[-0.35$, $0.20]$",-0.62,.809
LC_noun_M_verb_X,,noun_M - verb_X,-0.22,"$[-0.50$, $0.05]$",-1.90,.139
LF_noun_F_noun_M,LF,noun_F - noun_M,0.07,"$[-0.21$, $0.35]$",0.59,.827
LF_noun_F_verb_X,,noun_F - verb_X,0.02,"$[-0.26$, $0.29]$",0.15,.987
LF_noun_M_verb_X,,noun_M - verb_X,-0.05,"$[-0.33$, $0.23]$",-0.43,.901
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.35,"$[-0.63$, $-0.07]$",-2.97,.009
LP_noun_F_verb_X,,noun_F - verb_X,-0.13,"$[-0.41$, $0.15]$",-1.10,.513
LP_noun_M_verb_X,,noun_M - verb_X,0.22,"$[-0.06$, $0.50]$",1.87,.149


In [23]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 5 of 12

`tmin0==0.35 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,3.10,1.85,75.70,0.35,.055,.001
2,Chan0,73.46,2.40,98.51,20.13,< .001,.603
3,Cond0 $\times$ Chan0,1.93,4.76,195.08,0.80,.094,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,1.386136,2.801570,0.1764823
noun_M,252,1.328305,2.807033,0.1768265
verb_X,252,1.259814,2.792640,0.1759198


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,1.66,"$[1.12$, $2.20]$",6.03,< .001
noun_M_LC,,noun_M,1.64,"$[1.10$, $2.18]$",5.97,< .001
verb_X_LC,,verb_X,1.59,"$[1.05$, $2.13]$",5.77,< .001
noun_F_LF,LF,noun_F,-1.01,"$[-1.55$, $-0.46]$",-3.66,< .001
noun_M_LF,,noun_M,-1.05,"$[-1.59$, $-0.51]$",-3.82,< .001
verb_X_LF,,verb_X,-1.30,"$[-1.84$, $-0.76]$",-4.72,< .001
noun_F_LP,LP,noun_F,3.02,"$[2.48$, $3.56]$",10.97,< .001
noun_M_LP,,noun_M,3.18,"$[2.64$, $3.72]$",11.56,< .001
verb_X_LP,,verb_X,2.93,"$[2.39$, $3.48]$",10.67,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.02,"$[-0.30$, $0.33]$",0.14,.990
LC_noun_F_verb_X,,noun_F - verb_X,0.07,"$[-0.24$, $0.39]$",0.55,.849
LC_noun_M_verb_X,,noun_M - verb_X,0.05,"$[-0.26$, $0.37]$",0.41,.911
LF_noun_F_noun_M,LF,noun_F - noun_M,0.04,"$[-0.27$, $0.36]$",0.34,.939
LF_noun_F_verb_X,,noun_F - verb_X,0.29,"$[-0.02$, $0.61]$",2.20,.072
LF_noun_M_verb_X,,noun_M - verb_X,0.25,"$[-0.07$, $0.56]$",1.86,.151
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.16,"$[-0.47$, $0.15]$",-1.21,.446
LP_noun_F_verb_X,,noun_F - verb_X,0.08,"$[-0.23$, $0.40]$",0.63,.802
LP_noun_M_verb_X,,noun_M - verb_X,0.25,"$[-0.07$, $0.56]$",1.85,.156


In [24]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 6 of 12

`tmin0==0.35 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,9.56,1.61,66.12,0.56,.001,.004
2,Chan0,70.93,2.49,102.11,19.80,< .001,.594
3,Cond0 $\times$ Chan0,1.40,4.88,200.12,0.77,.228,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-0.5541810,2.813528,0.1772356
noun_M,252,-0.5114951,2.796140,0.1761403
verb_X,252,-0.3092414,2.780667,0.1751655


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,0.23,"$[-0.32$, $0.78]$",0.83,.408
noun_M_LC,,noun_M,0.34,"$[-0.20$, $0.89]$",1.23,.219
verb_X_LC,,verb_X,0.41,"$[-0.14$, $0.95]$",1.46,.145
noun_F_LF,LF,noun_F,-3.18,"$[-3.73$, $-2.63]$",-11.46,< .001
noun_M_LF,,noun_M,-3.09,"$[-3.64$, $-2.55]$",-11.15,< .001
verb_X_LF,,verb_X,-2.98,"$[-3.52$, $-2.43]$",-10.73,< .001
noun_F_LP,LP,noun_F,0.85,"$[0.31$, $1.40]$",3.08,.002
noun_M_LP,,noun_M,1.21,"$[0.66$, $1.76]$",4.36,< .001
verb_X_LP,,verb_X,1.29,"$[0.74$, $1.84]$",4.65,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.11,"$[-0.43$, $0.21]$",-0.83,.687
LC_noun_F_verb_X,,noun_F - verb_X,-0.18,"$[-0.50$, $0.14]$",-1.29,.400
LC_noun_M_verb_X,,noun_M - verb_X,-0.06,"$[-0.38$, $0.26]$",-0.47,.887
LF_noun_F_noun_M,LF,noun_F - noun_M,-0.09,"$[-0.41$, $0.23]$",-0.65,.793
LF_noun_F_verb_X,,noun_F - verb_X,-0.20,"$[-0.52$, $0.12]$",-1.50,.292
LF_noun_M_verb_X,,noun_M - verb_X,-0.12,"$[-0.44$, $0.20]$",-0.85,.672
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.35,"$[-0.67$, $-0.04]$",-2.61,.025
LP_noun_F_verb_X,,noun_F - verb_X,-0.44,"$[-0.76$, $-0.12]$",-3.20,.004
LP_noun_M_verb_X,,noun_M - verb_X,-0.08,"$[-0.40$, $0.24]$",-0.59,.824


In [25]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 7 of 12

`tmin0==0.41 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,0.65,1.68,69.06,0.50,.500,.000
2,Chan0,42.94,2.14,87.93,28.56,< .001,.471
3,Cond0 $\times$ Chan0,2.00,4.00,163.88,1.08,.097,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,1.396912,2.756034,0.1736138
noun_M,252,1.417533,2.670787,0.1682438
verb_X,252,1.352892,2.747269,0.1730617


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,1.62,"$[1.01$, $2.23]$",5.25,< .001
noun_M_LC,,noun_M,1.71,"$[1.11$, $2.32]$",5.55,< .001
verb_X_LC,,verb_X,1.62,"$[1.02$, $2.23]$",5.26,< .001
noun_F_LF,LF,noun_F,-0.82,"$[-1.43$, $-0.21]$",-2.65,.008
noun_M_LF,,noun_M,-0.93,"$[-1.54$, $-0.32]$",-3.01,.003
verb_X_LF,,verb_X,-1.03,"$[-1.63$, $-0.42]$",-3.33,.001
noun_F_LP,LP,noun_F,2.62,"$[2.02$, $3.23]$",8.50,< .001
noun_M_LP,,noun_M,2.93,"$[2.33$, $3.54]$",9.51,< .001
verb_X_LP,,verb_X,2.68,"$[2.07$, $3.29]$",8.68,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.09,"$[-0.43$, $0.24]$",-0.65,.790
LC_noun_F_verb_X,,noun_F - verb_X,0.00,"$[-0.34$, $0.33]$",-0.02,> .999
LC_noun_M_verb_X,,noun_M - verb_X,0.09,"$[-0.25$, $0.43]$",0.63,.801
LF_noun_F_noun_M,LF,noun_F - noun_M,0.11,"$[-0.23$, $0.45]$",0.76,.728
LF_noun_F_verb_X,,noun_F - verb_X,0.21,"$[-0.13$, $0.54]$",1.45,.316
LF_noun_M_verb_X,,noun_M - verb_X,0.10,"$[-0.24$, $0.44]$",0.69,.770
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.31,"$[-0.65$, $0.02]$",-2.18,.076
LP_noun_F_verb_X,,noun_F - verb_X,-0.05,"$[-0.39$, $0.28]$",-0.38,.923
LP_noun_M_verb_X,,noun_M - verb_X,0.26,"$[-0.08$, $0.59]$",1.79,.172


In [26]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 8 of 12

`tmin0==0.41 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,4.34,1.39,57.12,0.77,.029,.001
2,Chan0,42.48,2.02,82.78,30.59,< .001,.450
3,Cond0 $\times$ Chan0,1.86,4.05,166.14,1.10,.118,.003


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-0.9018101,2.813277,0.1772198
noun_M,252,-0.9031400,2.705026,0.1704006
verb_X,252,-0.7359581,2.830054,0.1782766


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,0.06,"$[-0.57$, $0.70]$",0.20,.841
noun_M_LC,,noun_M,0.08,"$[-0.56$, $0.71]$",0.24,.812
verb_X_LC,,verb_X,0.22,"$[-0.41$, $0.85]$",0.69,.490
noun_F_LF,LF,noun_F,-3.17,"$[-3.81$, $-2.54]$",-9.88,< .001
noun_M_LF,,noun_M,-3.38,"$[-4.01$, $-2.75]$",-10.52,< .001
verb_X_LF,,verb_X,-3.12,"$[-3.75$, $-2.49]$",-9.70,< .001
noun_F_LP,LP,noun_F,-0.23,"$[-0.86$, $0.41]$",-0.70,.484
noun_M_LP,,noun_M,0.21,"$[-0.42$, $0.84]$",0.66,.512
verb_X_LP,,verb_X,0.04,"$[-0.59$, $0.67]$",0.12,.905


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.01,"$[-0.36$, $0.34]$",-0.08,.996
LC_noun_F_verb_X,,noun_F - verb_X,-0.16,"$[-0.51$, $0.19]$",-1.07,.535
LC_noun_M_verb_X,,noun_M - verb_X,-0.15,"$[-0.49$, $0.20]$",-0.98,.587
LF_noun_F_noun_M,LF,noun_F - noun_M,0.21,"$[-0.14$, $0.56]$",1.41,.339
LF_noun_F_verb_X,,noun_F - verb_X,-0.06,"$[-0.40$, $0.29]$",-0.38,.925
LF_noun_M_verb_X,,noun_M - verb_X,-0.26,"$[-0.61$, $0.08]$",-1.78,.177
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.44,"$[-0.78$, $-0.09]$",-2.95,.009
LP_noun_F_verb_X,,noun_F - verb_X,-0.26,"$[-0.61$, $0.08]$",-1.78,.176
LP_noun_M_verb_X,,noun_M - verb_X,0.17,"$[-0.18$, $0.52]$",1.17,.474


In [27]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 9 of 12

`tmin0==0.6 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,6.80,1.65,67.56,0.63,.004,.003
2,Chan0,10.73,2.13,87.14,27.13,< .001,.185
3,Cond0 $\times$ Chan0,2.88,5.06,207.59,0.81,.015,.004


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,1.393528,2.173353,0.1369084
noun_M,252,1.242988,2.086968,0.1314666
verb_X,252,1.161003,2.071985,0.1305228


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,1.15,"$[0.57$, $1.74]$",3.88,< .001
noun_M_LC,,noun_M,1.06,"$[0.47$, $1.64]$",3.56,< .001
verb_X_LC,,verb_X,0.97,"$[0.39$, $1.56]$",3.28,.001
noun_F_LF,LF,noun_F,0.08,"$[-0.50$, $0.66]$",0.27,.788
noun_M_LF,,noun_M,-0.30,"$[-0.88$, $0.28]$",-1.01,.311
verb_X_LF,,verb_X,-0.52,"$[-1.11$, $0.06]$",-1.77,.078
noun_F_LP,LP,noun_F,0.71,"$[0.13$, $1.30]$",2.41,.017
noun_M_LP,,noun_M,0.80,"$[0.22$, $1.39]$",2.71,.007
verb_X_LP,,verb_X,0.65,"$[0.07$, $1.24]$",2.20,.028


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.10,"$[-0.24$, $0.43]$",0.67,.780
LC_noun_F_verb_X,,noun_F - verb_X,0.18,"$[-0.16$, $0.52]$",1.26,.421
LC_noun_M_verb_X,,noun_M - verb_X,0.08,"$[-0.25$, $0.42]$",0.58,.828
LF_noun_F_noun_M,LF,noun_F - noun_M,0.38,"$[0.04$, $0.72]$",2.66,.022
LF_noun_F_verb_X,,noun_F - verb_X,0.60,"$[0.27$, $0.94]$",4.23,< .001
LF_noun_M_verb_X,,noun_M - verb_X,0.22,"$[-0.11$, $0.56]$",1.57,.262
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.09,"$[-0.43$, $0.25]$",-0.63,.803
LP_noun_F_verb_X,,noun_F - verb_X,0.06,"$[-0.28$, $0.40]$",0.42,.908
LP_noun_M_verb_X,,noun_M - verb_X,0.15,"$[-0.19$, $0.49]$",1.05,.545


In [28]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 10 of 12

`tmin0==0.6 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,12.75,1.52,62.49,0.92,< .001,.006
2,Chan0,14.49,1.87,76.81,31.36,< .001,.222
3,Cond0 $\times$ Chan0,3.67,5.34,218.76,0.76,.003,.005


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-1.1442963,2.363520,0.1488878
noun_M,252,-1.0738460,2.170951,0.1367570
verb_X,252,-0.7880923,2.247977,0.1416092


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,-0.77,"$[-1.38$, $-0.16]$",-2.48,.014
noun_M_LC,,noun_M,-0.75,"$[-1.36$, $-0.14]$",-2.42,.016
verb_X_LC,,verb_X,-0.51,"$[-1.12$, $0.10]$",-1.64,.103
noun_F_LF,LF,noun_F,-2.47,"$[-3.08$, $-1.86]$",-7.95,< .001
noun_M_LF,,noun_M,-2.58,"$[-3.20$, $-1.97]$",-8.32,< .001
verb_X_LF,,verb_X,-2.44,"$[-3.06$, $-1.83]$",-7.87,< .001
noun_F_LP,LP,noun_F,-2.25,"$[-2.86$, $-1.64]$",-7.25,< .001
noun_M_LP,,noun_M,-1.75,"$[-2.36$, $-1.14]$",-5.63,< .001
verb_X_LP,,verb_X,-1.52,"$[-2.13$, $-0.91]$",-4.89,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,-0.02,"$[-0.37$, $0.33]$",-0.13,.991
LC_noun_F_verb_X,,noun_F - verb_X,-0.26,"$[-0.61$, $0.09]$",-1.77,.179
LC_noun_M_verb_X,,noun_M - verb_X,-0.24,"$[-0.59$, $0.10]$",-1.64,.229
LF_noun_F_noun_M,LF,noun_F - noun_M,0.12,"$[-0.23$, $0.46]$",0.79,.707
LF_noun_F_verb_X,,noun_F - verb_X,-0.02,"$[-0.37$, $0.32]$",-0.16,.987
LF_noun_M_verb_X,,noun_M - verb_X,-0.14,"$[-0.49$, $0.21]$",-0.95,.608
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.51,"$[-0.85$, $-0.16]$",-3.43,.002
LP_noun_F_verb_X,,noun_F - verb_X,-0.73,"$[-1.08$, $-0.39]$",-4.97,< .001
LP_noun_M_verb_X,,noun_M - verb_X,-0.23,"$[-0.57$, $0.12]$",-1.54,.272


In [29]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 11 of 12

`tmin0==0.11 & mode0=="pos"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.83,1.91,78.23,0.19,.169,.000
2,Chan0,80.22,1.65,67.68,26.15,< .001,.624
3,Cond0 $\times$ Chan0,1.00,5.10,209.18,0.59,.417,.001


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,1.100700,2.745554,0.1729536
noun_M,252,1.032498,2.706017,0.1704630
verb_X,252,1.047033,2.693244,0.1696584


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,0.11,"$[-0.40$, $0.63]$",0.44,.659
noun_M_LC,,noun_M,0.02,"$[-0.49$, $0.53]$",0.08,.933
verb_X_LC,,verb_X,-0.01,"$[-0.52$, $0.50]$",-0.03,.976
noun_F_LF,LF,noun_F,-0.73,"$[-1.24$, $-0.22]$",-2.81,.005
noun_M_LF,,noun_M,-1.03,"$[-1.55$, $-0.52]$",-3.99,< .001
verb_X_LF,,verb_X,-0.94,"$[-1.45$, $-0.43]$",-3.63,< .001
noun_F_LP,LP,noun_F,3.43,"$[2.92$, $3.94]$",13.23,< .001
noun_M_LP,,noun_M,3.54,"$[3.03$, $4.05]$",13.66,< .001
verb_X_LP,,verb_X,3.36,"$[2.84$, $3.87]$",12.94,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.09,"$[-0.18$, $0.36]$",0.80,.701
LC_noun_F_verb_X,,noun_F - verb_X,0.12,"$[-0.15$, $0.39]$",1.06,.540
LC_noun_M_verb_X,,noun_M - verb_X,0.03,"$[-0.24$, $0.30]$",0.26,.965
LF_noun_F_noun_M,LF,noun_F - noun_M,0.31,"$[0.04$, $0.58]$",2.66,.022
LF_noun_F_verb_X,,noun_F - verb_X,0.21,"$[-0.06$, $0.48]$",1.85,.156
LF_noun_M_verb_X,,noun_M - verb_X,-0.09,"$[-0.37$, $0.18]$",-0.81,.696
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.11,"$[-0.38$, $0.16]$",-0.98,.592
LP_noun_F_verb_X,,noun_F - verb_X,0.08,"$[-0.20$, $0.35]$",0.65,.790
LP_noun_M_verb_X,,noun_M - verb_X,0.19,"$[-0.08$, $0.46]$",1.63,.234


In [30]:
IDX <- IDX+1; COND <- CONDS[IDX]
data1 <- droplevels(subset(df1, eval(parse(text=COND)))); data1 <- data1[complete.cases(data1),]; done1 <- AOV4(data1,COND)
display_markdown(str_c("# Condition ",IDX," of ",length(CONDS))); display_markdown(done1$info); display_markdown("### Table"); done1$aov_APA$table; display_markdown("### Result"); display_markdown(done1$aov_APA[["full_result"]][["cond0"]]); display_markdown("### Summary (all done1, incl. NANs)"); done1$summary; display_markdown("### Expected marginal means"); done1$emm_APA$table; display_markdown("### Paired comparisons for expected marginal means"); done1$pairs_APA$table

# Condition 12 of 12

`tmin0==0.11 & mode0=="neg"`

### Table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,Cond0,1.03,1.85,75.96,0.21,.357,.000
2,Chan0,58.64,1.67,68.57,21.35,< .001,.552
3,Cond0 $\times$ Chan0,1.03,4.56,187.08,0.68,.397,.002


### Result

...

cond0,N,valX_M,valX_SD,valX_SE
<fct>,<int>,<dbl>,<dbl>,<dbl>
noun_F,252,-0.4305127,2.252176,0.1418737
noun_M,252,-0.3816280,2.276384,0.1433987
verb_X,252,-0.3821270,2.209403,0.1391793


### Expected marginal means

,chan0,cond0,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
noun_F_LC,LC,noun_F,-0.63,"$[-1.10$, $-0.17]$",-2.71,.007
noun_M_LC,,noun_M,-0.74,"$[-1.20$, $-0.28]$",-3.17,.002
verb_X_LC,,verb_X,-0.67,"$[-1.13$, $-0.21]$",-2.85,.005
noun_F_LF,LF,noun_F,-2.33,"$[-2.79$, $-1.87]$",-9.94,< .001
noun_M_LF,,noun_M,-2.33,"$[-2.79$, $-1.87]$",-9.93,< .001
verb_X_LF,,verb_X,-2.36,"$[-2.82$, $-1.89]$",-10.05,< .001
noun_F_LP,LP,noun_F,1.23,"$[0.77$, $1.69]$",5.25,< .001
noun_M_LP,,noun_M,1.37,"$[0.91$, $1.83]$",5.83,< .001
verb_X_LP,,verb_X,1.20,"$[0.74$, $1.66]$",5.13,< .001


### Paired comparisons for expected marginal means

,chan0,contrast,estimate,ci,statistic,p.value
,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
LC_noun_F_noun_M,LC,noun_F - noun_M,0.11,"$[-0.17$, $0.38]$",0.92,.629
LC_noun_F_verb_X,,noun_F - verb_X,0.03,"$[-0.24$, $0.31]$",0.28,.958
LC_noun_M_verb_X,,noun_M - verb_X,-0.07,"$[-0.35$, $0.20]$",-0.64,.799
LF_noun_F_noun_M,LF,noun_F - noun_M,0.00,"$[-0.28$, $0.27]$",-0.01,> .999
LF_noun_F_verb_X,,noun_F - verb_X,0.03,"$[-0.25$, $0.30]$",0.23,.971
LF_noun_M_verb_X,,noun_M - verb_X,0.03,"$[-0.25$, $0.30]$",0.24,.969
LP_noun_F_noun_M,LP,noun_F - noun_M,-0.14,"$[-0.41$, $0.14]$",-1.17,.471
LP_noun_F_verb_X,,noun_F - verb_X,0.03,"$[-0.25$, $0.30]$",0.24,.967
LP_noun_M_verb_X,,noun_M - verb_X,0.17,"$[-0.11$, $0.44]$",1.42,.333
